ΚΑΡΛΗΣ ΝΙΚΟΛΑΟΣ 1115201800068


ΧΑΤΖΗΠΑΥΛΟΥ ΜΑΡΙΑ 1115201400223

In [60]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [61]:
from keras.layers import Input, Dense, Conv1D, MaxPooling1D, UpSampling1D, BatchNormalization, LSTM, RepeatVector
from keras.models import Model
from keras.models import model_from_json
from keras import regularizers
import datetime
import time
import requests as req
import json
import pandas as pd
import pickle
import os
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
%pylab inline

Populating the interactive namespace from numpy and matplotlib


Διάβασμα του αρχείου με μετατροπή σε dataframe

In [62]:
# read files
df1 = pd.read_csv('/content/gdrive/MyDrive/nasd_input.csv', header=None,sep = '\t')

df1 = df.reset_index(drop=True)

print("Number of rows and columns:", df1.shape)


Number of rows and columns: (110, 731)


In [63]:
df2 = pd.read_csv('/content/gdrive/MyDrive/nasd_query.csv', header=None,sep = '\t')

df2 = df2.reset_index(drop=True)
print("Number of rows and columns:", df2.shape)

print(df2)

Number of rows and columns: (10, 731)
    0       1       2       3    ...       727       728       729      730
0   aer  39.170  39.120  39.140  ...   52.1900   51.9800   51.8700   51.670
1  aeri  30.760  29.910  28.260  ...   64.7500   63.8250   61.7500   62.000
2   aes  12.023  12.167  12.158  ...   10.5100   10.5100   10.3000   10.230
3   aet  88.070  87.655  86.723  ...  176.8300  175.4400  172.5300  175.120
4  aeti   5.730   5.990   5.700  ...    1.5499    1.5000    1.4499    1.400
5   aey   2.540   2.540   2.540  ...    1.4101    1.4000    1.4040    1.421
6  aezs  61.160  61.000  61.000  ...    1.8900    1.9000    1.9700    2.050
7  afam  29.850  29.710  29.590  ...   51.2000   53.9500   52.8500   51.250
8   afb  11.810  11.801  11.761  ...   13.6300   13.5900   13.5700   13.550
9   afc  20.596  20.630  20.736  ...   25.5600   25.7504   25.7475   25.625

[10 rows x 731 columns]


In [64]:
print(df1)

      0       1       2       3    ...      727      728      729     730
0       a  40.130  40.617  40.120  ...  68.2500  68.1100  67.4700  66.810
1      aa  34.711  34.283  34.260  ...  46.9700  45.8900  43.3300  43.010
2    aaba  50.880  51.150  50.020  ...  72.2200  71.1600  70.1900  70.560
3     aac  29.500  29.600  29.500  ...  11.0700  11.1100  10.8800  11.280
4     aal  49.295  49.422  48.811  ...  46.3580  46.2690  45.6700  45.820
..    ...     ...     ...     ...  ...      ...      ...      ...     ...
105   aey   2.540   2.540   2.540  ...   1.4101   1.4000   1.4040   1.421
106  aezs  61.160  61.000  61.000  ...   1.8900   1.9000   1.9700   2.050
107  afam  29.850  29.710  29.590  ...  51.2000  53.9500  52.8500  51.250
108   afb  11.810  11.801  11.761  ...  13.6300  13.5900  13.5700  13.550
109   afc  20.596  20.630  20.736  ...  25.5600  25.7504  25.7475  25.625

[110 rows x 731 columns]


In [65]:
frames = [df1, df2]
  
df= pd.concat(frames)
df = df.reset_index(drop=True)
print(df)

      0       1       2       3    ...      727      728      729     730
0       a  40.130  40.617  40.120  ...  68.2500  68.1100  67.4700  66.810
1      aa  34.711  34.283  34.260  ...  46.9700  45.8900  43.3300  43.010
2    aaba  50.880  51.150  50.020  ...  72.2200  71.1600  70.1900  70.560
3     aac  29.500  29.600  29.500  ...  11.0700  11.1100  10.8800  11.280
4     aal  49.295  49.422  48.811  ...  46.3580  46.2690  45.6700  45.820
..    ...     ...     ...     ...  ...      ...      ...      ...     ...
115   aey   2.540   2.540   2.540  ...   1.4101   1.4000   1.4040   1.421
116  aezs  61.160  61.000  61.000  ...   1.8900   1.9000   1.9700   2.050
117  afam  29.850  29.710  29.590  ...  51.2000  53.9500  52.8500  51.250
118   afb  11.810  11.801  11.761  ...  13.6300  13.5900  13.5700  13.550
119   afc  20.596  20.630  20.736  ...  25.5600  25.7504  25.7475  25.625

[120 rows x 731 columns]


το df.shape[0] είναι το πλήθος των γραμμών και το df.shape[1] είναι το πλήθος των στηλών. 

Αφού πρέπει να χωρίσουμε κάθε time series σε 80% training set και 20% test set βρίσκουμε σε ποιο column του dataframe αντιστοιχεί το 80% του time series που είναι το df.shape[1] * 0.8

Αποθηκεύουμε το αποτέλεσμα στην μεταβλητή percentage

In [66]:
rows = df.shape[0]
columns = df.shape[1] -1

percentage = int(columns * 0.80)

In [67]:
startdate="01/01/2015"
window_length = 10
encoding_dim = 3
epochs = 100
test_samples = 2000

In [68]:
def mkdate(ts):
    return datetime.datetime.fromtimestamp(
        int(ts)
    ).strftime('%Y-%m-%d')

def plot_examples(stock_input, stock_decoded):
    n = 10  
    plt.figure(figsize=(20, 4))
    for i, idx in enumerate(list(np.arange(0, test_samples, 200))):
        # display original
        ax = plt.subplot(2, n, i + 1)
        if i == 0:
            ax.set_ylabel("Input", fontweight=600)
        else:
            ax.get_yaxis().set_visible(False)
        plt.plot(stock_input[idx])
        ax.get_xaxis().set_visible(False)
        

        # display reconstruction
        ax = plt.subplot(2, n, i + 1 + n)
        if i == 0:
            ax.set_ylabel("Output", fontweight=600)
        else:
            ax.get_yaxis().set_visible(False)
        plt.plot(stock_decoded[idx])
        ax.get_xaxis().set_visible(False)
        
        
def plot_history(history):
    plt.figure(figsize=(15, 5))
    ax = plt.subplot(1, 2, 1)
    plt.plot(history.history["loss"])
    plt.title("Train loss")
    ax = plt.subplot(1, 2, 2)
    plt.plot(history.history["val_loss"])
    plt.title("Test loss")

Σκεφτήκαμε να εφαρμόσουμε τον κώδικα που μας δόθηκε αλλά αντί για ένα dataframe με μία στήλη με πολλές γραμμές έχουμε πολλές στήλες με πολλές γραμμές.


Ξεκινήσαμε να κάνουμε iterate κάθε γραμμή του dataframe με το iterrows(). Παρατηρήσαμε ότι με την χρήση του row εμφανίζονταν κάθετα τα δεδομένα της γραμμής, σαν στήλη, επομένως σε κάθε επανάληψη δημιουργούμε ένα temp dataframe που έχει τα time series κάθε γραμμής. 

Έτσι το dataframe temp είναι σαν αυτό που έχει στο παράδειγμα και μπορούμε δημιουργήσουμε σε κάθε επανάληψη τα αντίστοιχα X_train και X_test, και τα αποθηκεύουμε σε αντίστοιχες λίστες trainList και testList.

Το scale γίνεται ξεχωριστά σε κάθε γραμμή του dataframe

In [69]:

scaler = MinMaxScaler()

trainList = []
testList = []

for index, row in df.iterrows():
  temp = pd.DataFrame(row[1:].astype(float))
  temp.columns = ['price']

  temp['pct_change'] = temp.price.pct_change()
  temp['log_ret'] = np.log(temp.price) - np.log(temp.price.shift(1))
  x_train_nonscaled = np.array([temp['log_ret'].values[i-window_length:i].reshape(-1, 1) for i in tqdm(range(window_length+1,len(temp['log_ret'])))])
  x_train = np.array([scaler.fit_transform(temp['log_ret'].values[i-window_length:i].reshape(-1, 1)) for i in tqdm(range(window_length+1,len(temp['log_ret'])))])
  
  x_test = x_train[percentage:]
  x_train = x_train[:percentage]

  x_train = x_train.astype('float32')
  x_test = x_test.astype('float32')

  trainList.append(x_train)
  testList.append(x_test)

100%|██████████| 719/719 [00:00<00:00, 4796.68it/s]


Το μοντέλο που μας δόθηκε.

Αλλάξαμε το activation function σε tanh, διότι έχουμε κανονικοποιήσει τα δεδομένα με το minmaxscaler. Επομένως το tanh είναι καλύτερο από το sigmoid


In [70]:
input_window = Input(shape=(window_length,1))
x = Conv1D(16, 3, activation="tanh", padding="same")(input_window) # 10 dims
#x = BatchNormalization()(x)
x = MaxPooling1D(2, padding="same")(x) # 5 dims
x = Conv1D(1, 3, activation="tanh", padding="same")(x) # 5 dims
#x = BatchNormalization()(x)
encoded = MaxPooling1D(2, padding="same")(x) # 3 dims

encoder = Model(input_window, encoded)

# 3 dimensions in the encoded layer

x = Conv1D(1, 3, activation="tanh", padding="same")(encoded) # 3 dims
#x = BatchNormalization()(x)
x = UpSampling1D(2)(x) # 6 dims
x = Conv1D(16, 2, activation='tanh')(x) # 5 dims
#x = BatchNormalization()(x)
x = UpSampling1D(2)(x) # 10 dims
decoded = Conv1D(1, 3, activation='sigmoid', padding='same')(x) # 10 dims
autoencoder = Model(input_window, decoded)
autoencoder.summary()



Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 10, 1)]           0         
                                                                 
 conv1d_5 (Conv1D)           (None, 10, 16)            64        
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 5, 16)            0         
 1D)                                                             
                                                                 
 conv1d_6 (Conv1D)           (None, 5, 1)              49        
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 3, 1)             0         
 1D)                                                             
                                                                 
 conv1d_7 (Conv1D)           (None, 3, 1)              4   

In [ ]:
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')


for x_train, x_test in zip(trainList, testList): #10
  history = autoencoder.fit(x_train, x_train, epochs=10, batch_size=1024,
                shuffle=True, validation_data=(x_test, x_test))


Epoch 1/10
1/1 [==============================] - 1s 1s/step - loss: 0.6924 - val_loss: 0.6936
Epoch 2/10
1/1 [==============================] - 0s 42ms/step - loss: 0.6923 - val_loss: 0.6935
Epoch 3/10
1/1 [==============================] - 0s 43ms/step - loss: 0.6921 - val_loss: 0.6935
Epoch 4/10
1/1 [==============================] - 0s 43ms/step - loss: 0.6920 - val_loss: 0.6934
Epoch 5/10
1/1 [==============================] - 0s 38ms/step - loss: 0.6919 - val_loss: 0.6933
Epoch 6/10
1/1 [==============================] - 0s 56ms/step - loss: 0.6918 - val_loss: 0.6932
Epoch 7/10
1/1 [==============================] - 0s 44ms/step - loss: 0.6917 - val_loss: 0.6931
Epoch 8/10
1/1 [==============================] - 0s 51ms/step - loss: 0.6916 - val_loss: 0.6930
Epoch 9/10
1/1 [==============================] - 0s 43ms/step - loss: 0.6914 - val_loss: 0.6929
Epoch 10/10
1/1 [==============================] - 0s 44ms/step - loss: 0.6913 - val_loss: 0.6928
Epoch 1/10
1/1 [===============

Για κάθε X_test στο testList αποθηκεύουμε τα prediction του encoder σε μια encodedList

In [ ]:
encodedList = []
# print(len(decodedList)

for index, X_test in enumerate(testList):
    encoded_stocks = encoder.predict(X_test)
    encodedList.append(encoded_stocks)
    # predicted_stock_price = scaler.inverse_transform(decoded_stocks)

In [ ]:
# Calling `save('my_model')` creates a SavedModel folder `my_model`.
autoencoder.save('/content/gdrive/MyDrive/modeld')

INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/modeld/assets


In [ ]:
encoder.save('/content/gdrive/MyDrive/encoder')

INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/encoder/assets


In [71]:
encoder.load_weights('/content/gdrive/MyDrive/encoder')

Κάνουμε iterate την encodedList και παίρνουμε τα encoded time series τα κάνουμε dataframe μιας σειράς και το προσθέτουμε σε μια λίστα απο dataframe. Αυτή τη λίστα την κάνουμε το dataframe result.

Προσθέτουμε το id ως πρώτη στήλη χωρίζουμε το dataframe σε query και dataset και τα κάνουμε export ως .csv αρχεία

In [72]:
listDf = []

for d in encodedList:

  mylist = []
  for i_1 in range(d.shape[0]):
    for i_2 in range(d.shape[1]):
      mylist.append(d[i_1][i_2])

  temp = pd.DataFrame(mylist).transpose()

  listDf.append(temp)


In [80]:
result = pd.concat(listDf)
result = result.reset_index(drop=True)

In [81]:
result.insert(loc = 0, column = 'id', value= df.iloc[:, 0])
result.columns = [''] * len(result.columns)
print(result)

                                         ...                                        
0       a  0.241652  0.110029  0.160694  ...  0.110331 -0.013950  0.026331  0.149709
1      aa  0.221040  0.120105  0.378955  ... -0.011456  0.028372 -0.260263 -0.080608
2    aaba  0.265935  0.006023  0.024015  ...  0.134481 -0.194282  0.043780  0.243700
3     aac  0.335409  0.271537  0.196155  ...  0.286200  0.461384  0.192765  0.395947
4     aal -0.084710  0.157108  0.230467  ...  0.030228  0.366508  0.105200  0.256767
..    ...       ...       ...       ...  ...       ...       ...       ...       ...
105   aey -0.080252 -0.013075  0.103034  ... -0.054602  0.050219 -0.070907  0.010874
106  aezs -0.261176 -0.357976  0.022031  ... -0.027022  0.244079  0.267509  0.212249
107  afam  0.246135  0.132461  0.189068  ...  0.234429  0.354109  0.121585  0.293265
108   afb  0.183187  0.041062 -0.091509  ... -0.085917  0.254791  0.013267  0.028098
109   afc  0.117506  0.117564  0.033116  ... -0.014825  0.128155 

In [82]:
query = result.tail(10)
query = query.reset_index(drop=True)
print(query)

                                       ...                                        
0   aer  0.126035  0.062424  0.092664  ...  0.312029  0.055917  0.046276  0.203217
1  aeri  0.201087  0.081599  0.071118  ... -0.148869 -0.100151 -0.232410  0.054488
2   aes  0.160980  0.160206  0.208193  ... -0.023439  0.144759  0.036398  0.055199
3   aet  0.138326  0.112968  0.060063  ...  0.353579  0.381935 -0.019216  0.075908
4  aeti  0.251796  0.153412 -0.032648  ...  0.016443  0.308642  0.134867  0.166779
5   aey -0.080252 -0.013075  0.103034  ... -0.054602  0.050219 -0.070907  0.010874
6  aezs -0.261176 -0.357976  0.022031  ... -0.027022  0.244079  0.267509  0.212249
7  afam  0.246135  0.132461  0.189068  ...  0.234429  0.354109  0.121585  0.293265
8   afb  0.183187  0.041062 -0.091509  ... -0.085917  0.254791  0.013267  0.028098
9   afc  0.117506  0.117564  0.033116  ... -0.014825  0.128155 -0.017769 -0.140203

[10 rows x 406 columns]


In [83]:
dataset = result.head(100)
dataset = dataset.reset_index(drop=True)
print(dataset)

                                        ...                                        
0      a  0.241652  0.110029  0.160694  ...  0.110331 -0.013950  0.026331  0.149709
1     aa  0.221040  0.120105  0.378955  ... -0.011456  0.028372 -0.260263 -0.080608
2   aaba  0.265935  0.006023  0.024015  ...  0.134481 -0.194282  0.043780  0.243700
3    aac  0.335409  0.271537  0.196155  ...  0.286200  0.461384  0.192765  0.395947
4    aal -0.084710  0.157108  0.230467  ...  0.030228  0.366508  0.105200  0.256767
..   ...       ...       ...       ...  ...       ...       ...       ...       ...
95   aek  0.362141  0.081519  0.012836  ...  0.279390  0.144100  0.292913  0.177289
96   ael  0.153743  0.258215  0.291046  ... -0.073132  0.071853 -0.091466  0.175841
97   aem  0.291350  0.308198  0.163586  ...  0.216363  0.409645  0.364576 -0.005333
98   aeo  0.103685  0.344195  0.337120  ...  0.065043  0.322361  0.081354  0.108329
99   aep  0.340254  0.074690  0.223670  ...  0.089284  0.289953  0.365663  0

In [88]:
dataset.to_csv("/content/gdrive/MyDrive/datasetd.csv", sep="\t", index=False, header=False)

In [87]:
query.to_csv("/content/gdrive/MyDrive/queryd.csv",  sep="\t", index=False, header=False)